In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import yaml

from typing import Dict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

import optuna
from optuna.trial import TrialState

%load_ext kedro.ipython
device = torch.device('mps')

/Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[02/25/24 19:38:59] INFO     Resolved project path as:                                              ]8;id=961380;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=111863;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#139\139]8;;\
                             /Users/gavinlou/Developer/kedro/ML-GS/gas-sensor-ml.                                  
                             To set a different path, run '%reload_kedro <project_root>'                           

[02/25/24 19:38:59] INFO     Kedro project gas-sensor-ml                                            ]8;id=745680;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=664474;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#108\108]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=381864;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=425525;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#109\109]8;;\
                             'pipelines'                                                                           

[02/25/24 19:39:00] INFO     Registered line magic 'run_viz'                                        ]8;id=702231;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=279911;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#115\115]8;;\

In [139]:
%reload_ext kedro.ipython

[01/24/24 17:30:06] INFO     Resolved project path as:                                              ]8;id=431689;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=109207;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#139\139]8;;\
                             /Users/gavinlou/Developer/kedro/ML-GS/gas-sensor-ml.                                  
                             To set a different path, run '%reload_kedro <project_root>'                           

[01/24/24 17:30:06] INFO     Kedro project gas-sensor-ml                                            ]8;id=897890;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=199637;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#108\108]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=844221;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=861336;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#109\109]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=956487;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=308203;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#115\115]8;;\

In [2]:
mox_bin = catalog.load("mox_bin")
# mox_bin.tail()
lstm_params = catalog.load("params:complete_lstm_model.model_options")
model_input_table = catalog.load("model_input_table")

[02/25/24 19:39:13] INFO     Loading data from 'mox_bin' (ParquetDataset)...                    ]8;id=379908;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=313425;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'params:complete_lstm_model.model_options'       ]8;id=439048;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=886335;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from 'model_input_table' (ParquetDataset)...          ]8;id=908920;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=801222;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

In [3]:
study_params = catalog.load("params:complete_lstm_model.study_options")

[02/25/24 19:39:17] INFO     Loading data from 'params:complete_lstm_model.study_options'       ]8;id=602220;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=649335;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\
                             (MemoryDataset)...                                                                    

In [4]:
print(lstm_params)
print(lstm_params['num_epochs'])

{'num_classes': 1, 'num_epochs': 10, 'batch_size': 32, 'learning_rate': 0.001, 'test_size': 0.2, 'random_state': 3, 'input_size': 1, 'sequence_length': 750, 'hidden_size': 128, 'num_layers': 2, 'val_size': 0.2}
10


In [5]:

# Hyper-parameters 

num_classes = lstm_params['num_classes']
num_epochs = lstm_params['num_epochs']
batch_size = lstm_params['batch_size']
learning_rate = lstm_params['learning_rate']

"""

Each feature as a time step in your sequence, you could set sequence_length to 150 and input_size to 1.
This would mean you are feeding in sequences of length 150, with each time step in the sequence having 1 feature.

"""

input_size = lstm_params['input_size']
sequence_length = lstm_params['sequence_length'] # the window it trains with can be selected
hidden_size = lstm_params['hidden_size']
num_layers = lstm_params['num_layers']
random_state = lstm_params['random_state']

# batch_size = lstm_params['batch_size']  # You can adjust the batch size according to your needs

test_size = lstm_params['test_size']
val_size = lstm_params['val_size']

In [6]:
# NaN processing
# forward fill NaN values
def _ffill_NaN (X_dataset: np.ndarray) -> np.ndarray:
    X_dataset_df = pd.DataFrame(X_dataset)
    X_dataset_df.ffill(inplace=True)
    # Convert back to numpy arrays
    return X_dataset

In [7]:
# Clean NaN values from input table
_ffill_NaN(model_input_table)
# check for NaN values
model_input_table.isnull().sum()


bin_0        0
bin_1        0
bin_2        0
bin_3        0
bin_4        0
            ..
bin_746      0
bin_747      0
bin_748      0
bin_749      0
res_ratio    0
Length: 751, dtype: int64

In [8]:
# Implement LSTM functions below
# there is no validation set in this example
# load mox_table as input

def split_data(model_input_table: pd.DataFrame) -> torch.tensor:
    # Split data into features and target
    X = model_input_table[model_input_table.columns[:-1]].values  # Assuming last column is the target
    y = model_input_table[model_input_table.columns[-1]].values
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size = test_size, random_state = random_state)
    
    # Further split to create a validation set
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, 
        test_size = val_size, 
        random_state = random_state)
    
    # Initialize StandardScaler
    scaler = StandardScaler()
    # Fit on training data
    scaler.fit(X_train)

    # Transform both training and testing data
    X_train_scaled = scaler.transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    # Ensure y_train and y_test are in the correct format
    if isinstance(y_train, pd.Series):
        y_train = y_train.values
    if isinstance(y_val, pd.Series):
        y_val = y_val.values
    if isinstance(y_test, pd.Series):
        y_test = y_test.values

    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train_scaled.astype(np.float32))
    y_train_tensor = torch.tensor(y_train.astype(np.float32))

    X_val_tensor = torch.tensor(X_val_scaled.astype(np.float32))
    y_val_tensor = torch.tensor(y_val.astype(np.float32))

    X_test_tensor = torch.tensor(X_test_scaled.astype(np.float32))
    y_test_tensor = torch.tensor(y_test.astype(np.float32))

    return X_train_tensor, X_val_tensor, X_test_tensor, y_train_tensor, y_val_tensor, y_test_tensor


# create X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor from split_data(df)
X_train_tensor, X_val_tensor, X_test_tensor, y_train_tensor, y_val_tensor, y_test_tensor = split_data(model_input_table)
# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
# Initialize DataLoaders
# batch_size = lstm_params['batch_size']  # You can adjust the batch size according to your needs
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)


In [9]:
# OPTUNA model

# Optuna managed model
class RNN(nn.Module):
    # def __init__(self, input_size, hidden_size, num_layers, num_classes):
    def __init__(self, trial, input_size, num_classes):
        super(RNN, self).__init__()

        # Optuna suggests the number of layers and hidden size
        self.num_layers = trial.suggest_int("num_layers", 1, 3)
        self.hidden_size = trial.suggest_int("hidden_size", 30, 100)

        # Optuna suggests the dropout ratio of each layer
        dropout_rate = trial.suggest_float("dropout_rate", 0, 0.5)

        # Define the LSTM layer
        self.lstm = nn.LSTM(
            input_size, 
            self.hidden_size, 
            self.num_layers, 
            batch_first=True,
            dropout=(dropout_rate if self.num_layers > 1 else 0), 
            )
        self.fc = nn.Linear(self.hidden_size, num_classes)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        out, _ = self.lstm(x, (h0,c0))  
        out = out[:, -1, :]
        out = self.fc(out)
        return out

def define_model(trial, input_size, num_classes):
    model = RNN(trial, input_size, num_classes)
    return model

In [10]:
# Optuna managed training
def objective(trial, input_size, num_classes):
    model = define_model(trial, input_size, num_classes).to(device)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(torch.optim, optimizer_name)(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    # Training of the model
    n_total_steps = len(train_loader)
    for epoch in range(num_epochs):
        model.train()

        for i, (bins, target) in enumerate(train_loader):
            bins = bins.reshape(-1, sequence_length, input_size).to(device)
            target = target.squeeze().to(device)

            # Forward pass
            outputs = model(bins)
            # Example of reshaping/squeezing if applicable
            outputs = outputs.squeeze()  # Removes dimensions of size 1
            # outputs = outputs[:64]  # Adjust if you need to slice the outputs
            # target = target.squeeze().to(device)  # Add an extra dimension to match outputs
            loss = criterion(outputs, target)
        
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():  # Don't calculate gradients
            total_loss = 0
            count = 0

            for bins, target in test_loader:  # Replace with your validation loader
                bins = bins.reshape(-1, sequence_length, input_size).to(device)
                target = target.squeeze().to(device)  # Add an extra dimension to match outputs
                
                outputs = model(bins)
                outputs = outputs.squeeze()

                loss = criterion(outputs, target)
                total_loss += loss.item()
                count += 1
        
        model.train() # Set the model back to training mode

        rmse = np.sqrt(total_loss / count)
        trial.report(rmse, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
        
    return rmse
            # print(f'Epoch [{epoch+1}/{num_epochs}], RMSE on validation data: {rmse}')


In [12]:
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    # study.optimize(objective, n_trials=5, timeout=600)
    study.optimize(lambda trial: objective(trial, input_size, num_classes), n_trials=100, timeout=3600)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2024-02-25 19:51:09,549] A new study created in memory with name: no-name-77094495-34aa-4492-b5ee-a9eb9a9a4b67
[I 2024-02-25 19:51:53,916] Trial 0 finished with value: 0.0745268976554694 and parameters: {'num_layers': 1, 'hidden_size': 92, 'dropout_rate': 0.05801663533524276, 'optimizer': 'RMSprop', 'lr': 0.0022304195118295544}. Best is trial 0 with value: 0.0745268976554694.
[I 2024-02-25 19:53:14,866] Trial 1 finished with value: 0.2965646622211691 and parameters: {'num_layers': 3, 'hidden_size': 46, 'dropout_rate': 0.21404039084181753, 'optimizer': 'SGD', 'lr': 0.00010237593144469875}. Best is trial 0 with value: 0.0745268976554694.
[I 2024-02-25 19:55:48,217] Trial 2 finished with value: 0.4032772426305503 and parameters: {'num_layers': 3, 'hidden_size': 99, 'dropout_rate': 0.4904259339261965, 'optimizer': 'SGD', 'lr': 5.703293945545027e-05}. Best is trial 0 with value: 0.0745268976554694.
[I 2024-02-25 19:58:26,947] Trial 3 finished with value: 0.0836666740221685 and parameters

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  74
  Number of complete trials:  26
Best trial:
  Value:  0.06789462219246084
  Params: 
    num_layers: 1
    hidden_size: 69
    dropout_rate: 0.47486099740715404
    optimizer: Adam
    lr: 0.0051787606123568026


# Optuna Visualisation with RDB backend

In [13]:
storage_name = "sqlite:///your_study.db"  # Name your database file
study_name = "your_study_name"  # Assign a name to your study
new_study = optuna.create_study(direction="minimize", study_name=study_name, storage=storage_name, load_if_exists=True)


[I 2024-02-25 20:21:00,908] Using an existing study with name 'your_study_name' instead of creating a new one.


In [14]:
# transfer trials from memory to sqlite database
for trial in study.trials:
    new_study.add_trial(trial)

In [15]:
# verify that the trials were added
print("Original study trials:", len(study.trials))
print("New study trials:", len(new_study.trials))

Original study trials: 100
New study trials: 110


In [16]:
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_rank
from optuna.visualization import plot_slice
from optuna.visualization import plot_timeline

In [18]:
plot_optimization_history(study)
plot_intermediate_values(study)


In [19]:
plot_timeline(study)

[02/25/24 20:29:52] WARNING  /var/folders/ty/_fq55jvd4533rn8vbpz3_kzc0000gn/T/ipykernel_43008/54762 ]8;id=799361;file:///Users/gavinlou/.pyenv/versions/3.10.13/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=309695;file:///Users/gavinlou/.pyenv/versions/3.10.13/lib/python3.10/warnings.py#109\109]8;;\
                             9393.py:1: ExperimentalWarning:                                                       
                                                                                                                   
                             plot_timeline is experimental (supported from v3.2.0). The interface                  
                             can change in the future.                                                             
                                                                                                                   
                                                                                                                   